# Overview
The notebook packages up a pretrained ResNet50 model and exports it as javascript, based loosely on the 

In [ ]:
import argparse
import importlib.util
import inspect
import os
import sys
sys.setrecursionlimit(10000)  # workaround for deep copying large graph
import traceback
from os import path

import keras

from webdnn.backend import generate_descriptor
from webdnn.frontend.keras import KerasConverter
from webdnn.graph import traverse
from webdnn.graph.placeholder import Placeholder
from webdnn.graph.shape import Shape
from webdnn.graph.traverse import dump_dot
from webdnn.util import flags, console

In [ ]:
from keras.applications import ResNet50
pretrained_model = ResNet50(weights = 'imagenet', include_top = True)
pretrained_model.save('resnet50.h5')

In [ ]:
input_shapes = [Shape.parse('(1, 299, 299, 3)')[0]]
model_path = 'resnet50.h5'
model = keras.models.load_model(model_path, 
                                custom_objects=[], compile=False)

In [ ]:
model.build(input_shape=None)
converter = KerasConverter(batch_size=input_shapes[0][0])
graph = converter.convert(model)
traverse.dump(graph)

In [ ]:
for graph_input, input_shape in zip(graph.inputs, input_shapes):
    for p1, p2 in zip(graph_input.shape, input_shape):
        if not Placeholder.check_resolved(p1) and Placeholder.check_resolved(p2):
            p1.value = Placeholder.force_int(p2)

        elif Placeholder.check_resolved(p1) and not Placeholder.check_resolved(p2):
            raise ValueError(f'Shape mismatch: expected:{input_shape}, real:{graph_input.shape}, {p1} != {p2}')

        elif Placeholder.check_resolved(p1) and Placeholder.check_resolved(p2):
            assert p1 == p2, f'Shape mismatch: expected:{input_shape}, real:{graph_input.shape}, {p1} != {p2}'

output_dir = path.join(path.dirname(model_path), "webdnn_graph_descriptor")
os.makedirs(output_dir, exist_ok=True)

ir_dot_path = path.join(output_dir, "ir.dot")
with open(ir_dot_path, "w") as f:
    f.write(dump_dot(graph))
console.stderr(f"IR graph can be visualized with graphviz command: 'dot {ir_dot_path} -T png -o output.png'")

console.stderr(f" Generating graph descriptor")

In [ ]:
!dot {ir_dot_path} -T png -o output.png

In [ ]:
any_backend_failed = False
backends = 'webgpu,webgl,webassembly,fallback'.split(',')
for i, backend in enumerate(backends):
    console.stderr(f"BackendName: {console.colorize(backend, console.Color.Cyan)}")
    try:
        graph_exec_data = generate_descriptor(backend, graph, constant_encoder_name=None)
        graph_exec_data.save(output_dir)
    except Exception as ex:
        if flags.DEBUG:
            raise ex

        any_backend_failed = True
        console.error(f"Failed generating descriptor for {backend} backend")
        console.stderr(traceback.format_exc())
        continue

if any_backend_failed:
    raise ValueError('BackendFailed')
    # raise last_backend_exception